In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import pandas as pd
import joblib
import json
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("..")

/mnt/data/stephana93dm/storage/conda_envs/alternative_clustering/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [7]:
from alternative_image_clustering.paper.load_benchmark_data import (
    create_kmeans_row,
    create_nr_baseline_row,
    create_per_prompt_rows,
    create_table_head,
)

base_dir = "/mnt/data/stephana93dm/storage/projects/alternative_image_clustering"

datasets = ["fruit360", "gtsrb", "nrobjects", "cards"]

metrics = ["ACC", "AMI"]

rows = [
    create_kmeans_row(base_dir, "image", datasets, metrics),
    create_nr_baseline_row(base_dir, "image", "nrkmeans", datasets, metrics),
]

for embedding_type in ["tfidf", "sbert_concat"]:
    rows.extend(create_per_prompt_rows(base_dir, embedding_type, datasets, metrics))
    rows.append(create_kmeans_row(base_dir, embedding_type, datasets, metrics, "per_category_kmeans"))
    rows.append(create_kmeans_row(base_dir, embedding_type, datasets, metrics, "full_kmeans"))

    rows.append(create_nr_baseline_row(base_dir, embedding_type, "nrkmeans", datasets, metrics))


df = pd.DataFrame(rows, columns=pd.MultiIndex.from_tuples(create_table_head(base_dir, datasets, metrics)))
# df[df.columns[2:]] = (df[df.columns[2:]] * 100).round(2)
df.set_index(pd.MultiIndex.from_arrays([df.values[:, 0], df.values[:, 1]]), inplace=True)

df = df[df.columns[2:]]
df = (df*100).round(2)

In [14]:
df.values[:, 0:2]

array([['image', 'kmeans'],
       ['image', 'nrkmeans'],
       ['tfidf', 'per_prompt'],
       ['tfidf', 'per_prompt_max'],
       ['tfidf', 'per_category_kmeans'],
       ['tfidf', 'full_kmeans'],
       ['tfidf', 'nrkmeans'],
       ['sbert_concat', 'per_prompt'],
       ['sbert_concat', 'per_prompt_max'],
       ['sbert_concat', 'per_category_kmeans'],
       ['sbert_concat', 'full_kmeans'],
       ['sbert_concat', 'nrkmeans']], dtype=object)

In [3]:
df.set_index(pd.MultiIndex.from_arrays([df.values[:, 0], df.values[:, 1]]), inplace=True)

In [6]:
df[df.columns[2:]]

fruit360                       gtsrb         \
                                    fruit        colour          type          
                                      ACC    AMI    ACC    AMI    ACC    AMI   
image        kmeans                 53.14  41.28  56.15  47.29  61.20  51.55   
             nrkmeans               62.97  50.61  58.69  55.47  58.33  38.62   
tfidf        per_prompt             48.85  24.78  62.31  50.67  67.94  52.37   
             per_prompt_max         55.75  34.77  72.71  67.38  73.00  58.36   
             per_category_kmeans    53.37  32.22  69.36  65.53  78.50  67.93   
             full_kmeans            51.08  26.40  60.66  56.02  76.52  68.93   
             nrkmeans               49.47  30.05  62.83  55.35  78.50  62.59   
sbert_concat per_prompt             48.30  25.01  66.27  56.92  67.70  55.54   
             per_prompt_max         56.67  35.93  71.38  68.81  74.26  61.01   
             per_category_kmeans    51.66  27.64  72.00  62.58  76.18  62.56   
             full_kmeans            47.22  25.23  56.91  47.54  73.30  64.89   
             nrkmeans               50.01  29.32  67.98  58.65  74.80  36.92   

                                               nrobjects                   \
                                 colour            shape         material   
                                    ACC    AMI       ACC     AMI      ACC   
image        kmeans               73.99  33.37     96.66   96.22    50.53   
             nrkmeans             68.40  43.25     82.41   82.22    62.99   
tfidf        per_prompt           93.47  73.68     82.38   71.31    61.41   
             per_prompt_max       96.95  81.76     99.86   99.17    68.73   
             per_category_kmeans  96.76  82.33     95.49   96.52    56.35   
             full_kmeans          89.35  70.44     93.46   91.60    50.97   
             nrkmeans             84.06  53.46     82.44   84.04    56.47   
sbert_concat per_prompt           97.04  82.75     86.77   79.36    63.56   
             per_prompt_max       97.71  85.30     99.99   99.92    77.71   
             per_category_kmeans  97.43  84.49    100.00  100.00    51.27   
             full_kmeans          86.67  63.20     92.12   92.31    50.98   
             nrkmeans             72.04  59.86     71.67   75.29    62.51   

                                                       cards                \
                                        colour          rank          suit   
                                    AMI    ACC    AMI    ACC    AMI    ACC   
image        kmeans                0.00  30.74  21.73  41.04  47.83  51.90   
             nrkmeans             32.68  61.27  65.68  45.65  55.18  33.12   
tfidf        per_prompt           10.07  79.95  81.20  39.59  41.32  51.79   
             per_prompt_max       20.07  82.94  86.15  42.20  48.44  56.13   
             per_category_kmeans   1.77  93.92  95.27  40.66  41.61  61.05   
             full_kmeans           0.02  25.90  13.25  39.06  39.85  59.18   
             nrkmeans             12.98  52.89  55.16  42.14  38.04  45.56   
sbert_concat per_prompt           15.02  88.09  88.28  47.19  49.99  57.07   
             per_prompt_max       40.56  96.22  95.16  51.21  56.09  62.35   
             per_category_kmeans   0.06  84.75  88.73  69.74  70.34  57.28   
             full_kmeans           0.02  23.57   4.80  50.19  54.08  55.69   
             nrkmeans             32.02  27.85  12.24  66.13  68.39  52.18   

                                         
                                         
                                    AMI  
image        kmeans               34.36  
             nrkmeans             16.57  
tfidf        per_prompt           27.38  
             per_prompt_max       36.65  
             per_category_kmeans  37.08  
             full_kmeans          34.92  
             nrkmeans             23.86  
sbert_concat per_prompt           33.63  
             per_prompt_max       37.32  
 